In [15]:
import pandas as pd

In [16]:
ResOCTWS = pd.read_csv('./res/OCTWS.csv')
ResOCTNoWS = pd.read_csv('./res/OCTNoWS.csv')
ResCART = pd.read_csv('./res/CART.csv')
res = {'OCTWS':ResOCTWS, 'OCTNoWS':ResOCTNoWS, 'CART':ResCART}

In [17]:
def resProcess(res):
    """
    Calculate the average results from random data split
    Select alpha with highest validation accuracy
    """
    if 'alpha' in res.columns:
        # Calculate the mean of the feat for each instance, depth and alpha
        mean = res.groupby(['instance', 'depth']).mean().reset_index()
        std = res.groupby(['instance', 'depth']).std().reset_index()
    else:
        mean = res.groupby(['instance', 'depth']).mean().reset_index()
        std = res.groupby(['instance', 'depth']).std().reset_index()
    return mean, std

In [18]:
def resCombine(res, feat):
    df = pd.DataFrame(columns=['instance', 'depth'])
    for r in res:
        df_mean, df_std = resProcess(res[r])
        df_mean = df_mean[['instance', 'depth', feat]]
        df_std = df_std[['instance', 'depth', feat]]
        if r == "CART":
            df_mean[feat] = df_mean[feat].round(4)
            df_std[feat] = df_std[feat].round(4)
        else:
            df_mean[feat] = df_mean[feat].round(1)
            df_std[feat] = df_std[feat].round(1)
        df_mean[feat] = df_mean[feat].astype(str)
        df_std[feat] = df_std[feat].astype(str)
        df = df.merge(df_mean, how='outer', on=['instance', 'depth'])
        df = df.merge(df_std, how='outer', on=['instance', 'depth'])
        # rename columns to distinguish different models
        df.rename(columns={feat+'_x':r+'_mean', feat+'_y':r+'_std'}, inplace=True)
        df[r] = df[r+"_mean"] + " (" + df[r+'_std'] + ")"
    df = df[['instance', 'depth'] + list(res.keys())]
    return df

In [19]:
resCombine(res, 'train_time').to_csv('./res/ResTrainTime.csv', index=False)